# Plant growth: Carbon assimilation, respiration, and allocation

<div class="alert alert-warning">
If figures/tables are not rendered correctly, make sure your notebook is `trusted` by choosing `Trust Notebook` command found in `View` - `Activate Command Palette` menu. Typing in the first letters (i.e. "tr") should give you the command.
</div>

<div class="alert alert-info">
To run the entire code at once, go to 'Run' menu and select 'Run All Cells'. This should run all cells in sequential order. 

Once a cell is run, you will see a number indicating the order a cell has run in brackets like [1]. If you see [*] next to a cell, that means the cell is still running. 

After all cells are run, you can use the sliders to evaluate the effects of environmental factors and plant traits on whole-plant growth and biomass. If plots are unresponsive to sliders, save the notebook, and refresh the webpage with your browser.


Predict and observe how the model responds the changes you make. 
</div>

### Set up the notebook

In [ ]:
using Cropbox

In [ ]:
Cropbox.Interact.WebIO.setup(:ijulia)

In [ ]:
using Unitful

Using `Unitful` package, we can simply indicate the unit for each quantity and the package will take care of the most unit conversion and simplifcations for us. We use a pipe operator `|>` to force a conversion to the unit we want. We are creating a `let` block here which should end with `end`. In `Julia`, a `let` block contains the scope of variables and operations within.

### Problem 1. How much biomass does our sunflower gain in a day? ###
Apparently, your sunflower plants get bigger every week you measure them. How fast do they grow? Let's do some quick calculations of how many grams of biomass our sunflower plant would gain in a day. For simplicity, we will assume that the plant is growing in a growth chamber (similar to the greenhouse condition in our lab experiment but more controlled environments) with some additional simplifyng assumptions.

First we gather some information we need to do this. What do we need to know?

- light intensity as photosynthetic photon flux density (PFD) in the growth chamber: 500 μmol/m2/s
- photoperiod (DL): 12 hours per day 
- current biomass (W): 10g
- current leaf area (LA): 1000 cm2 
- specific leaf area (SLA): 200 cm2/g
- Photosynthesis at saturating light level (A_sat; photosynthetic capacity) with light saturation point at 1500 μmol/m^2/s : 30 μmol/m2/s at PFD of 1500 μmol/m^2/s or greater
- net CO2 assimilation rate (A) at the growing light level: 10 μmol/m2/s assuming a linear relationship between A and PFD up to the light saturation point.
- respiration rate (R): 1.5 μmol/m^2/s (area based) or 75 μmol/g/s (mass based)
- carbon concentration (w/w) in sunflower dry biomass (c_conc): 45%
- Molecular weights of C and CO2: 12g/mol and 44g/mol

Assumptions:
- Light is the only environmental factor that affect photosynthesis
- All leaves experience the same light intensity
- Respiration rate is the same for all plant tissue
- Notice other hidden assumptions we failed to spell out?

What is LAR (leaf area ratio) for this plant? What is LMR (leaf mass ratio)?

How much biomass(g) would this sunflower plant gain over a day? How about over a week? How about over 100 days? Can we just multiply the daily value by the number of days?


In [ ]:
# set some constants
C_mw, CO2_mw, C_conc = 12.0u"g/mol", 44.0u"g/mol", 0.45

In [ ]:
let
    W = plant_mass= 10.0u"g"    
    LA = 1000.0u"cm^2"
    SLA = 200.0u"cm^2/g"
    LMA = 1/SLA |> u"g/m^2" #leaf mass per area = 1/SLA
    LAR = LA/W |> u"m^2/g" 
    leaf_mass = LA*LMA
    LMR = leaf_mass/plant_mass 
    
    A = 10.0u"μmol/m^2/s"*SLA |> u"μmol/g/d" #area basis -> mass basis
    R = 1.5u"μmol/m^2/s"*SLA |> u"μmol/g/d"  #area basis -> mass basis
    DL = 12u"hr"
    CO2_gain = (A+R)*leaf_mass*(DL/24.0u"hr") |> u"μmol*d^-1"
    CO2_loss = R*W
    net_CO2_gain = CO2_gain - CO2_loss
    W_gain = net_CO2_gain*C_mw/C_conc |> u"g/d"
    
    println("Daily CO2 gain (μmol) for this plant is: ", CO2_gain)
    println("Daily CO2 loss (μmol) for this plant is: ", CO2_loss)
    println("Daily CO2 gain (g) for this plant is: ", CO2_gain*C_mw |> u"g/d")
    println("Daily CO2 loss (g) for this plant is: ", CO2_loss*C_mw |> u"g/d")
    println("LAR for this plant is: ", LAR |> u"m^2/g")
    println("LMA (g/m^2; 1/SLA) for this plant is: ", LMA |> u"g/m^2")
    println("LMR for this plant is: ", LMR |> u"g/g") 
    println("Fraction of CO2 loss through respiration per day is: ",  CO2_loss/CO2_gain)
    println("Daily Biomass gain (g/d) is: ", W_gain)
end   

### Carbon partitioning and plant growth

Allocation or partitioning of the photosynthates is a critical element to determine their destination and fate as to how and in what part they will be used. For example, some assimilates will be transported to roots to function as subsrates for providing ATPs to maintain the ion gradients while others can be invested back to build more new leaves. More investment in photosynthetic machinery (i.e. allocation to foliage) will yield greater return of biomass at a faster rate; this is similar to the growth of bank account balance with the compound interest rate. 

Monsi (1960) model effectively demonstrates the importance of photosynthate allocation in plant growth.

$$
r = k (m A - n R)
$$
  
This model is simple but elegant and is robustly grounded in the ecophysiology to provide critical insights for understanding the intricate relationships between the processes of photosynthesis, respiration, and carbon allocation, and their trade-offs determining the growth of plants. The **tradeoffs** of allocating photosynthates between photosynthetic and non-photosynthetic parts can be particularly insightful and relevant for the growth of crops like lettuce and cabbage for which the foliage biomass accounts for the majority of harvested yield or crops with large underground storage organs like radish, potato, garlic, onions, and taro. Carbon allocation to non-photosynthetic parts is also functionally important in  herbaceous perennials with storage organs such as rhizomes and woody plants for which dead parts (i.e. wood and bark) continue to be part of the whole-plant biomass.

### Problem 2. What will be the biomass of our sunflower plants after a month of growth? 
Using the same plant traits and growing conditions from Problem 1. Let's estimate expected plant biomass a month from now, and evaluate how the future biomass will change with changes in plant traits and growing conditions.   


- Table 4.1.

| Symbol | Value | Units | Description |
| :- | :- | :- | :- |
| $t$ | - | $\mathrm{d}$ | Time unit used in the model |
| $k$ | 0.0395 | - | Conversion factor from $\mathrm{CO_2}$ to biomass assuming 45% carbon in biomass |
| $m$ | 0.5 | - | Fraction of photosynthetic biomass |
| $n$ | 0.5 | - | Fraction of non-photosynthetic biomass ($1 - m$) |
| $A_{\mathrm{pot}}$ | 30.0 | $$\mathrm{\mu mol}\ \mathrm{m^{-2}}\ \mathrm{s^{-1}}$$ | Potential net $\mathrm{CO_2}$ assimilation rate (area basis) |
| $A_{\mathrm{sat}}$ | - | $$\mathrm{\mu mol}\ \mathrm{m^{-2}}\ \mathrm{s^{-1}}$$ | net $\mathrm{CO_2}$ assimilation rate at saturating light |
| $R_d$ | - | $\mathrm{\mu mol}\ \mathrm{m^{-2}}\ \mathrm{s^{-1}}$ | leaf dark respiration rate (area basis)|
| $\alpha$ | 0.03 | $\mathrm{d^{-1}}$ | Decay rate of the photosynthetic efficiency over time (reduction/day) |
| $A_{net}$ | - | $$\mathrm{\mu mol}\ \mathrm{m^{-2}}\ \mathrm{s^{-1}}$$ | Actual net $\mathrm{CO_2}$ assimilation rate during photoperiod |
| $R$ | - | $$\mathrm{\mu mol}\ \mathrm{m^{-2}}\ \mathrm{s^{-1}}$$ | Actual respiration rate over|
| $D_L$ | 12 | $\mathrm{hr}$ | Day length (i.e. photoperiod) |
| $D$ | 24 | $\mathrm{hr}$ | Total hours per day |
| $r$ | - | $\mathrm{g}$ $\mathrm{g^{-1}}$ $\mathrm{d^{-1}}$ | Intrinsic relative growth rate |
| $W_1$ | 10.0 | $\mathrm{g}$ | biomass at time 1 |
| $W$ | - | $\mathrm{g}$ | Resulting whole-plant biomass as dry matter  |

In [ ]:
@system Monsi1960a(Controller) begin
    t(context.clock.time) ~ track(u"d")
    
    A_pot: potential_gross_CO2_assimilation_at_saturating_PFD => 30.0 ~ preserve(parameter, u"μmol/m^2/s") # leaf area basis
    A_sat(A_pot): actual_gross_CO2_assimilation_at_saturating_PFD => A_pot ~ track(u"μmol/m^2/s") # leaf area basis
    r_cost: respiration_fraction_to_photosynthetic_capacity => 0.05 ~ preserve(parameter)
    Rd(A_sat, r_cost): dark_respiration => r_cost*A_sat ~ track(u"μmol/m^2/s") #leaf area basis
    
    PFD_sat: light_saturation_point => 1500.0 ~ preserve(parameter, u"μmol/m^2/s")
    PFD_growth: growing_light_level => 500.0 ~ preserve(parameter, u"μmol/m^2/s")
    QE(A_sat, PFD_sat): quantum_efficiency => A_sat/PFD_sat ~ track(u"μmol/μmol")

    A_net(QE, PFD_growth, A_sat, Rd): net_CO2_assimilation_rate_at_growing_light => begin
        min(QE*PFD_growth, A_sat) - Rd
    end ~ track(u"μmol/m^2/s")

    C_conc: biomass_carbon_content => 0.45 ~ preserve(parameter)
    k(C_conc): conversion_factor => ((12/44)/1000000*(60*60*24))/C_conc ~ preserve(parameter, u"g/g/μmol*s*m^2")    
    DL: day_length  => 12 ~ preserve(parameter, u"hr")
    D:  total_hours => 24 ~ preserve(parameter, u"hr")

    m:    leaf_mass_ratio   => 0.5 ~ preserve(parameter)
    n(m): non_leaf_fraction => 1-m ~ track

    A(DL, D, A_net): assimilation => A_net*(DL/D)  ~ track(u"μmol/m^2/s")
    R(Rd): respiration  => Rd   ~ track(u"μmol/m^2/s")


    r(k, m, n, A, R, D, DL): RGR => k * (m*A - n*R) ~ track(u"g/g/d")

    W1:      biomass_at_time_1 => 10.0 ~ preserve(parameter, u"g")
    W(r, W): biomass_at_time_2         => r*W  ~ accumulate(init = W1, u"g")
end

In [ ]:
c1 = @config (
    :Clock => (;
        step = 1u"d",
        stop = 20u"d",
        ),
    :Monsi1960a => (;
        W1 = 10.0,
        m = 0.5,
        A_pot = 30.0,
        DL = 12,
        ),
    )

In [ ]:
manipulate(Monsi1960a, :t, :W;
    config =  c1, stop = 30u"d",
    parameters = (
        :Monsi1960a =>
        (
            :W0    => 0:0.05:20.0,        
            :m    => 0:0.05:1,        
            :DL     => 0:1:24,
            :A_pot   => 0:1:60,
        ),
    ),
    xlim = (0, 30),
    ylim = (0, 100),    
    kind = :line,
)

In [ ]:
m60a = simulate(Monsi1960a, config = c1, stop = 30u"d", snap = 30u"d");

### Problem 3. Exponential plant growth is unrealistic. How can we make it more realistic?
Hypothesis: As plants age, the photosynthetic capacity ($A_{pot}$) decreases over time.

In [ ]:
@system Monsi1960b(Monsi1960a, Controller) begin
    α:    photosynthetic_capacity_decay_rate           => 0.03  ~ preserve(parameter, u"d^-1")
    LMA: leaf_mass_per_area               => 50.0  ~ preserve(parameter, u"g/m^2")        
    LA(W_l,LMA):  leaf_area_per_plant     => W_l/LMA      ~ preserve(parameter, u"m^2") 
    W_l(m,W): leaf_mass_at_time2          => m*W      ~ track(u"g")           

    A_sat(α, A_sat): actual_gross_CO2_assimilation_at_saturating_PFD => begin 
        -α*A_sat 
    end ~ accumulate(init=A_pot, u"μmol/m^2/s")

    A_net(QE, PFD_growth, A_sat, Rd): net_CO2_assimilation_rate_at_growing_light => begin
        min(QE*PFD_growth, A_sat) - Rd
    end ~ track(u"μmol/m^2/s")

    A(DL, D, A_net): assimilation => A_net*(DL/D)       ~ track(u"μmol/m^2/s")    
    R(Rd): respiration                    => Rd         ~ track(u"μmol/m^2/s")

    r(k, m, n, A, R): RGR => k * (m*A - n*R) ~ track(u"g/g/d")

    W1:      biomass_at_time_1 => 10.0 ~ preserve(parameter, u"g")
    W(r, W): biomass_at_time_2         => r*W  ~ accumulate(init = W1, u"g")
end

In [ ]:
visualize(Monsi1960b, :t, :W, config=c1, stop = 100u"d",  
    xlim = (0, 100),
    ylim = (0, 100),    
    kind=:line
)

In [ ]:
manipulate(Monsi1960b, :t, :W;
    config =  :Clock => :step => 1u"hr", stop = 100u"d",
    parameters = 
    (
    :Monsi1960b => 
        (
        :A_pot => 0:0.5:60,
        :r_cost => 0.0:0.001:0.1,
        :DL => 0:1:24,
        :m => 0:0.1:1.0,
        :α => 0:0.01:0.06,
        ),
    ),
    xlim = (0,100),
    ylim = (0,100),
    kind = :line,
)